# `function` 和 `method`

## `function`, `method` 和 `builtin_function_or_method`

与类和实例无绑定关系的 function 都属于 `函数（function）`，否则属于 `方法（method）`。

| 访问方式 | 函数类型 | 返回类型 | 是否描述符 | 绑定状态 |
|:---------|:---------|:---------|:---------|:---------|
| `standalone_func` | 独立函数 | `<class 'function'>` | 是 | 还未绑定 |
| `Class.method` | 实例方法 | `<class 'function'>` | 是 | 还未绑定 |
| `instance.method` | 实例方法 | `<class 'method'>` | 否 | 绑定到实例 |
| `Class.static_method` | 静态方法 | `<class 'function'>` | 是 | 无绑定 |
| `instance.static_method` | 静态方法 | `<class 'function'>` | 是 | 无绑定 |
| `Class.class_method` | 类方法 | `<class 'method'>` | 否 | 绑定到类 |
| `instance.class_method` | 类方法 | `<class 'method'>` | 否 | 绑定到类 |
| `lambda x: x` | Lambda函数 | `<class 'function'>` | 是 | 无绑定 |
| `len`, `print`, `max` | 内置函数 | `<class 'builtin_function_or_method'>` | 否 | C层自动处理 |
| `__new__` | 静态方法 | `<class 'builtin_function_or_method'>` | 否 | C层自动处理 |
| `instance.__init_` | 特殊的实例方法 | `<class 'wrapper_descriptor'>` | 否 | 绑定到实例 |

| 特性 | `<class 'function'>` | `<class 'method'>` |
|:-----|:---------------------|:-------------------|
| 调用方式 | 直接通过函数名调用 | 也是直接通过函数名调用（`对象.方法名`是构建一个`method`） |
| 参数传递 | 需要显式传递所有参数 | 自动传递绑定对象作为第一个参数 |
| 独立性 | 完全独立，不依赖任何对象 | 依赖于绑定的对象（类或实例） |

## `function` 都是描述符即实现了 `__get__`
`<class 'function'>` 是在 CPython 中的 C 代码中实现的，其 Python 等价实现：
```python
class Function:
    def __init__(self, func_code, func_name):
        self.__code__ = func_code
        self.__name__ = func_name
    
    # 绑定
    def __get__(self, instance, owner=None):
        """函数的描述符协议实现"""
        if instance is None:
            # 从类访问：返回未绑定的函数
            return self
        else:
            # 从实例访问：创建绑定方法
            import types
            return types.MethodType(self, instance)
    
    # 实现
    def __call__(self, *args, **kwargs):
        # 函数调用逻辑
        pass

class TestClass:
    def my_method(self, x):
        return f"处理: {x}"

obj = TestClass()

# 手动调用函数的 __get__ 方法
func = TestClass.my_method  # 这是一个 function 对象
print(f"原始函数类型: {type(func)}")  # <class 'function'>

# 手动调用 __get__ 模拟从类访问
result1 = func.__get__(None, TestClass)
print(f"从类访问结果: {type(result1)}")  # <class 'function'>
```

核心思想就是 `function` 类的两个方法：
- `绑定 __get__`（绑定到某个具体对象，实例或类）
- `实现 __call__`（该函数的具体实现逻辑）

## 将 `function` 绑定为 `method`（`method` 不是描述符）
使用 `types.MethodType` 可以将 `<class 'function'>` 变成了 `<class 'method'>`。

`Method` 对象拥有：
- `__self__`（绑定的对象）
- `__func__`（原始的被绑定的 `Function`）
### 基本转换示例
```python
import types

def standalone_function(self, x):
    return f"处理 {x} 来自 {self}"

class MyClass:
    def __init__(self, name):
        self.name = name

obj = MyClass("测试对象")

print("=== 转换前后的类型对比 ===")
print(f"原始函数类型: {type(standalone_function)}")  # <class 'function'>

# 使用 MethodType 绑定到实例
bound_method = types.MethodType(standalone_function, obj)
print(f"绑定后类型: {type(bound_method)}")  # <class 'method'>

# 调用对比
print(f"直接调用函数: {standalone_function(obj, 'data1')}")
print(f"调用绑定方法: {bound_method('data2')}")  # 自动传递 obj
```
### `Method` 对象的内部结构
```python
import types

def my_function(self, value):
    return f"函数处理: {value}"

class TestClass:
    def __init__(self, name):
        self.name = name

obj = TestClass("示例")
bound_method = types.MethodType(my_function, obj)

print("=== Method 对象的内部结构 ===")
print(f"__self__ (绑定的对象): {bound_method.__self__}")    # __self__ (绑定的对象): <__main__.TestClass object at 0x0000017D03647FD0>
print(f"__func__ (原始函数): {bound_method.__func__}")  # __func__ (原始函数): <function my_function at 0x0000017D0366A440>
print(f"__class__ (对象类型): {bound_method.__class__}")    # __class__ (对象类型): <class 'method'>

# 验证绑定关系
print(f"绑定的对象是否是 obj: {bound_method.__self__ is obj}")  # 绑定的对象是否是 obj: True
print(f"原始函数是否是 my_function: {bound_method.__func__ is my_function}")    # 原始函数是否是 my_function: True
```
### 不同绑定类型的对比
```python
import types

def test_function(cls_or_self, data):
    if isinstance(cls_or_self, type):
        return f"类方法处理: {data} (类: {cls_or_self.__name__})"
    else:
        return f"实例方法处理: {data} (实例: {cls_or_self})"

class MyClass:
    def __init__(self, name):
        self.name = name
    
    def __str__(self):
        return f"MyClass({self.name})"

obj = MyClass("测试")

print("=== 不同绑定类型 ===")

# 1. 绑定到实例 (实例方法)
instance_method = types.MethodType(test_function, obj)
print(f"实例绑定: {type(instance_method)}")  # 实例绑定: <class 'method'>
print(f"调用结果: {instance_method('数据1')}") # 调用结果: 实例方法处理: 数据1 (实例: MyClass(测试))

# 2. 绑定到类 (类方法)
class_method = types.MethodType(test_function, MyClass) 
print(f"类绑定: {type(class_method)}")  ## 类绑定: <class 'method'>
print(f"调用结果: {class_method('数据2')}") # 调用结果: 类方法处理: 数据2 (类: MyClass)
```


## `classmethod` 的Python等价实现
根据 Python 官方文档，`classmethod` 的纯Python等价实现是：
```python
class ClassMethod:
    def __init__(self, f):
        self.__func__ = f

    def __get__(self, obj, cls=None):
        if cls is None:
            cls = type(obj)
        # 如果 self.f 是个描述符即 function 对象，用其自身的 __get__ 绑定
        if hasattr(type(self.__func__), '__get__'):
            return self.__func__.__get__(cls, type(cls))
        return MethodType(self.__func__, cls)
```
即 `classmethod` 的核心思想当使用 `__get__` 对自身存储的 `self.__func__` 进行绑定时，一定是绑定到类而非实例上。
- 考虑
    ```python
    @classmethod
    def func():...
    ```
    等价于
    ```python 
    func = classmethod(func)    # 构建描述符对象（func 拥有了 self.__func__　属性）
    ```
- 而
    ```python
    Class/obj.func()
    ```
    等价于
    ```python 
    func.__get__(None/obj, Class/type(obj)).__call__() # 先绑定再运行
    ```

## `staticmethod` 的Python等价实现
根据 Python 官方文档，`staticmethod` 的纯Python等价实现是：
```python
class staticmethod:
    def __init__(self, func):
        self.func = func
    
    def __get__(self, instance, owner):
        # 静态方法不绑定实例或类，直接返回原函数
        return self.func
```

## 代码分析：`class_or_instancemethod`
```python
class class_or_instancemethod(classmethod):
    def __get__(self, instance: object, owner: tp.Optional[tp.Type] = None) -> tp.Any:
        descr_get = super().__get__ if instance is None else self.__func__.__get__
        return descr_get(instance, owner)
```

- 考虑
    ```python
    @class_or_instancemethod
    def func():...
    ```
    等价于
    ```python 
    func = class_or_instancemethod(func) # 构建描述符对象（func 拥有了 self.__func__　属性）
    ```
- 而
    ```python
    Class/obj.func()
    ```
    等价于
    ```python 
    func.__get__(None/obj, Class/type(obj)).__call__() # 先绑定再运行
    ```
    其中 
    ```python
    func.__get__(None/obj, Class/type(obj)) # 返回的是 <class 'method'> 对象
    ```
    会根据 `instance` 是否为空来决定将 `func` 绑定到 `Class` 或 `obj` 上。

